In [13]:
import pandas as pd
import ast
import os

In [15]:
# ---------- Utility Functions ----------

def safe_eval(val):
    """Safely parse stringified Python lists/dicts."""
    try:
        return ast.literal_eval(val)
    except:
        return []

def overlap(a_start, a_end, b_start, b_end, threshold=0.1):
    """Partial span overlap check: proportion of overlap >= threshold."""
    inter = max(0, min(a_end, b_end) - max(a_start, b_start))
    gt_len = max(1, a_end - a_start)
    pred_len = max(1, b_end - b_start)
    return inter / min(gt_len, pred_len) >= threshold

def get_list(val):
    """Convert comma-separated string into clean lowercase list."""
    if not val or str(val).lower() in ['na', 'none']:
        return []
    return [v.strip().lower() for v in str(val).split(',') if v.strip()]

def match_entities(gt_entities, pred_entities, threshold=0.1):
    """Match entities using partial span overlap."""
    matched_pairs = []
    used_pred = set()
    for gt in gt_entities:
        if len(gt) < 6:
            continue
        gt_entity, gt_s, gt_e, *_ = gt
        for i, pred in enumerate(pred_entities):
            if i in used_pred or len(pred) < 6:
                continue
            pred_entity, ps, pe, *_ = pred
            if overlap(gt_s, gt_e, ps, pe, threshold):
                matched_pairs.append((gt, pred))
                used_pred.add(i)
                break
    return matched_pairs

# ---------- Main Evaluation ----------

def evaluate_entity_level(csv_path, overlap_threshold=0.1):
    """Compute overall Precision, Recall, F1 for adj/adp/is_main_product."""
    df = pd.read_csv(csv_path)

    adj_tp = adj_fp = adj_fn = 0
    adp_tp = adp_fp = adp_fn = 0
    main_tp = main_fp = main_fn = 0

    for _, row in df.iterrows():
        gt_entities = safe_eval(row['gt'])
        pred_entities = safe_eval(row['pred'])
        matched_pairs = match_entities(gt_entities, pred_entities, threshold=overlap_threshold)

        for gt, pred in matched_pairs:
            gt_adj, gt_adp, gt_main = gt[3], gt[4], gt[5]
            pred_adj, pred_adp, pred_main = pred[3], pred[4], pred[5]

            gt_adj_list, pred_adj_list = get_list(gt_adj), get_list(pred_adj)
            gt_adp_list, pred_adp_list = get_list(gt_adp), get_list(pred_adp)

            # ---- adj ----
            if not gt_adj_list and not pred_adj_list:
                adj_tp += 1
            else:
                adj_tp += len(set(gt_adj_list) & set(pred_adj_list))
                adj_fp += len(set(pred_adj_list) - set(gt_adj_list))
                adj_fn += len(set(gt_adj_list) - set(pred_adj_list))

            # ---- adp ----
            if not gt_adp_list and not pred_adp_list:
                adp_tp += 1
            else:
                adp_tp += len(set(gt_adp_list) & set(pred_adp_list))
                adp_fp += len(set(pred_adp_list) - set(gt_adp_list))
                adp_fn += len(set(gt_adp_list) - set(pred_adp_list))

            # ---- is_main_product ----
            gt_main_bool = bool(gt_main) if str(gt_main).lower() not in ['na', 'none'] else False
            pred_main_bool = bool(pred_main) if str(pred_main).lower() not in ['na', 'none'] else False
            if gt_main_bool == pred_main_bool:
                main_tp += 1
            elif not gt_main_bool and pred_main_bool:
                main_fp += 1
            else:
                main_fn += 1

    def prf(tp, fp, fn):
        p = tp / (tp + fp) if (tp + fp) else 0
        r = tp / (tp + fn) if (tp + fn) else 0
        f1 = 2 * p * r / (p + r) if (p + r) else 0
        return p, r, f1

    adj_p, adj_r, adj_f1 = prf(adj_tp, adj_fp, adj_fn)
    adp_p, adp_r, adp_f1 = prf(adp_tp, adp_fp, adp_fn)
    main_p, main_r, main_f1 = prf(main_tp, main_fp, main_fn)

    metrics_df = pd.DataFrame({
        'Metric': ['adj', 'adp', 'is_main_product'],
        'Precision': [adj_p, adp_p, main_p],
        'Recall': [adj_r, adp_r, main_r],
        'F1': [adj_f1, adp_f1, main_f1]
    })
    return metrics_df

# ---------- Detailed CSV Generation ----------

def generate_detailed_csvs(csv_path, output_dir="./", overlap_threshold=0.1):
    df = pd.read_csv(csv_path)
    adj_rows, adp_rows, main_rows = [], [], []

    for _, row in df.iterrows():
        query = row['Query']
        gt_entities = safe_eval(row['gt'])
        pred_entities = safe_eval(row['pred'])
        matched_pairs = match_entities(gt_entities, pred_entities, threshold=overlap_threshold)

        # --- matched ---
        for gt, pred in matched_pairs:
            gt_entity, _, _, gt_adj, gt_adp, gt_main = gt
            pred_entity, _, _, pred_adj, pred_adp, pred_main = pred

            gt_adj_list, pred_adj_list = get_list(gt_adj), get_list(pred_adj)
            gt_adp_list, pred_adp_list = get_list(gt_adp), get_list(pred_adp)

            # ---- adj ----
            if not gt_adj_list and not pred_adj_list:
                adj_rows.append([query, gt_entity, pred_entity, None, "TP"])
            else:
                for adj in set(gt_adj_list) & set(pred_adj_list):
                    adj_rows.append([query, gt_entity, pred_entity, adj, "TP"])
                for adj in set(pred_adj_list) - set(gt_adj_list):
                    adj_rows.append([query, gt_entity, pred_entity, adj, "FP"])
                for adj in set(gt_adj_list) - set(pred_adj_list):
                    adj_rows.append([query, gt_entity, pred_entity, adj, "FN"])

            # ---- adp ----
            if not gt_adp_list and not pred_adp_list:
                adp_rows.append([query, gt_entity, pred_entity, None, "TP"])
            else:
                for adp in set(gt_adp_list) & set(pred_adp_list):
                    adp_rows.append([query, gt_entity, pred_entity, adp, "TP"])
                for adp in set(pred_adp_list) - set(gt_adp_list):
                    adp_rows.append([query, gt_entity, pred_entity, adp, "FP"])
                for adp in set(gt_adp_list) - set(pred_adp_list):
                    adp_rows.append([query, gt_entity, pred_entity, adp, "FN"])

            # ---- is_main_product ----
            gt_main_bool = bool(gt_main) if str(gt_main).lower() not in ['na', 'none'] else False
            pred_main_bool = bool(pred_main) if str(pred_main).lower() not in ['na', 'none'] else False
            status = "TP" if gt_main_bool == pred_main_bool else ("FP" if pred_main_bool else "FN")
            main_rows.append([query, gt_entity, pred_entity, "is_main_product", status])

        # --- unmatched preds (FP entities) ---
        for pred in pred_entities:
            if len(pred) < 6 or any(pred is p for _, p in matched_pairs): continue
            pred_entity = pred[0]
            adj_rows.append([query, None, pred_entity, None, "FP_entity"])
            adp_rows.append([query, None, pred_entity, None, "FP_entity"])
            main_rows.append([query, None, pred_entity, "is_main_product", "FP_entity"])

        # --- unmatched gts (FN entities) ---
        for gt in gt_entities:
            if len(gt) < 6 or any(gt is g for g, _ in matched_pairs): continue
            gt_entity = gt[0]
            adj_rows.append([query, gt_entity, None, None, "FN_entity"])
            adp_rows.append([query, gt_entity, None, None, "FN_entity"])
            main_rows.append([query, gt_entity, None, "is_main_product", "FN_entity"])

    # --- Save results ---
    adj_df = pd.DataFrame(adj_rows, columns=["query", "entity_gt", "entity_pred", "adj", "status"])
    adp_df = pd.DataFrame(adp_rows, columns=["query", "entity_gt", "entity_pred", "adp", "status"])
    main_df = pd.DataFrame(main_rows, columns=["query", "entity_gt", "entity_pred", "feature", "status"])

    adj_path = f"{output_dir.rstrip('/')}/adj.csv"
    adp_path = f"{output_dir.rstrip('/')}/adp.csv"
    main_path = f"{output_dir.rstrip('/')}/is_main_product.csv"

    adj_df.to_csv(adj_path, index=False)
    adp_df.to_csv(adp_path, index=False)
    main_df.to_csv(main_path, index=False)

    print(f"✅ Saved detailed CSVs:\n  - {adj_path}\n  - {adp_path}\n  - {main_path}")
    return adj_df, adp_df, main_df

In [23]:
# ---------- Example Usage ----------

if __name__ == "__main__":
    csv_path = "sample_input.csv"  # <-- change this
    output_dir = "eval_output"

    print("🔍 Evaluating metrics...")
    metrics = evaluate_entity_level(csv_path, overlap_threshold=0.1)
    print(metrics)

    print("\n📄 Generating detailed CSVs...")

    os.makedirs(output_dir, exist_ok=True)
    generate_detailed_csvs(csv_path, output_dir=output_dir, overlap_threshold=0.1)

🔍 Evaluating metrics...
            Metric  Precision    Recall        F1
0              adj   0.625000  0.714286  0.666667
1              adp   0.692308  0.818182  0.750000
2  is_main_product   1.000000  1.000000  1.000000

📄 Generating detailed CSVs...
✅ Saved detailed CSVs:
  - eval_output/adj.csv
  - eval_output/adp.csv
  - eval_output/is_main_product.csv
